# Capítulo 7 - Hipótese e inferência

In [1]:
from typing import Tuple, List
import math
import random

## Exemplo: lançando uma moeda

In [2]:
def aproximacao_normal_para_binomial(n: int, p: float) -> Tuple[float, float]:
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def probabilidade_normal_acima(lo: float, mu: float = 0, sigma: float = 1) -> float:
    return 1 - normal_cdf(lo, mu, sigma)

def probabilidade_normal_intermediario(lo: float, hi: float, mu: float = 0, sigma: float = 1) -> float:
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

def probabilidade_normal_fora(lo: float, hi: float, mu: float = 0, sigma: float = 1) -> float:
    return 1 - probabilidade_normal_intermediario(lo, hi, mu, sigma)

In [3]:
def normal_inversa_cdf(p: float, mu: float = 0, sigma: float = 1, tolerancia: float = 0.00001) -> float:
    if mu != 0 or sigma != 1:
        return mu + sigma * normal_inversa_cdf(p, tolerancia = tolerancia)
    low_z = -10
    hi_z = 10
    while hi_z - low_z > tolerancia:
        mid_z = (low_z + hi_z) / 2
        mid_p = normal_cdf(mid_z)
        if mid_p < p:
            low_z = mid_z
        else:
            hi_z = mid_z
    return mid_z

def normal_limite_superior(probabilidade: float, mu: float = 0, sigma: float = 1) -> float:
    return normal_inversa_cdf(probabilidade, mu, sigma)

def normal_limite_inferior(probabilidade: float, mu: float = 0, sigma: float = 1) -> float:
    return normal_inversa_cdf(1 - probabilidade, mu, sigma)

def normal_dois_lados(probabilidade: float, mu: float = 0, sigma: float = 1) -> Tuple[float, float]:
    probabilidade_cauda = (1 - probabilidade) / 2
    limite_superior = normal_limite_inferior(probabilidade_cauda, mu, sigma)
    limite_inferior = normal_limite_superior(probabilidade_cauda, mu, sigma)
    return limite_inferior, limite_superior

In [4]:
mu_0, sigma_0 = aproximacao_normal_para_binomial(1000, 0.5)
print(mu_0, sigma_0)

500.0 15.811388300841896


In [5]:
limite_inferior, limite_superior = normal_dois_lados(0.95, mu_0, sigma_0)
print(limite_inferior, limite_superior)

469.01026640487555 530.9897335951244


In [6]:
lo, hi = normal_dois_lados(0.95, mu_0, sigma_0)
mu_1, sigma_1 = aproximacao_normal_para_binomial(1000, 0.55)
probabilidade_tipo_2 = probabilidade_normal_intermediario(lo, hi, mu_1, sigma_1)
power = 1 - probabilidade_tipo_2
power

0.886548001295367

In [7]:
hi = normal_limite_superior(0.95, mu_0, sigma_0)
hi

526.0073585242053

In [8]:
probabilidade_tipo_2 = normal_cdf(hi, mu_1, sigma_1)
power = 1 - probabilidade_tipo_2
power

0.9363794803307173

## P-valor

In [9]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    if x >= mu:
        return 2 * probabilidade_normal_acima(x, mu, sigma)
    else:
        return 2 * normal_cdf(x, mu, sigma)
    
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

In [10]:
extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0
                    for _ in range(1000))
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1

print(59 < extreme_value_count < 65)

True


In [11]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

In [12]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
sigma

0.015791611697353755

In [13]:
normal_dois_lados(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

In [14]:
def run_experiment() -> List[bool]:
    return [random.random() < 0.5 for _ in range(1000)]

def rejeita_experimento(experimento: List[bool]) -> bool:
    num_heads = len([flip for flip in experimento if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experimentos = [run_experiment() for _ in range(1000)]
num_rejeicoes = len([experimento for experimento in experimentos if rejeita_experimento(experimento)])
print(num_rejeicoes == 46)

True


In [16]:
def parametro_estimado(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

def teste_estatistico_a_b(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = parametro_estimado(N_A, n_A)
    p_B, sigma_B = parametro_estimado(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

z = teste_estatistico_a_b(1000, 200, 1000, 180)
z

-1.1403464899034472

In [17]:
two_sided_p_value(z)

0.2541419765422359

In [18]:
z = teste_estatistico_a_b(1000, 200, 1000, 150)
z

-2.948839123097944

In [19]:
two_sided_p_value(z)

0.003189699706216853

## Inferência Bayesiana

In [ ]:
def B(alpha: float, beta: float) -> float:
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)
